In [3]:
%pip install --upgrade --quiet langchain
!pip install -qU langchain-openai
!pip install -qU pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.6 MB/s eta 0:00:00


In [2]:
import getpass
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


Tool calling

We use the term "tool calling" interchangeably with "function calling". Although function calling is sometimes meant to refer to invocations of a single function, we treat all models as though they can return multiple tool or function calls in each message.


Request: Passing tools to model

For a model to be able to invoke tools, you need to pass tool schemas to it when making a chat request. LangChain ChatModels supporting tool calling features implement a .bind_tools method, which receives a list of LangChain tool objects, Pydantic classes, or JSON Schemas and binds them to the chat model in the provider-specific expected format. Subsequent invocations of the bound chat model will include tool schemas in every call to the model API.
Defining tool schemas: LangChain Tool
For example, we can define the schema for custom tools using the @tool decorator on Python functions:

In [4]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# Define simple mathematical tools: addition and multiplication
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

# Store tools in a list
tools = [add, multiply]

In [5]:
# Initialize the OpenAI model (GPT-3.5-turbo in this case)
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Bind tools to the model
llm_with_tools = llm.bind_tools(tools)

In [6]:
# Define your query to trigger tool usage
query = "What is 3 * 12? Also, what is 11 + 49?"

# Invoke the model with the query
ai_message = llm_with_tools.invoke(query)

# Output the tool calls made by the model
print("Tool calls made by the model:", ai_message.tool_calls)

Tool calls made by the model: [{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_jzMBfdhav67wXBUDA92tZvb9', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_rNTbb5YcvCB2G8yTSVfgSWw8', 'type': 'tool_call'}]


In [8]:
from langchain_core.messages import HumanMessage, ToolMessage

# Initialize conversation with the human query
messages = [HumanMessage(query)]
messages.append(ai_message)

# Call the tools and pass their output back to the model
for tool_call in ai_message.tool_calls:
    # Determine which tool to use
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]

    # Call the tool with the provided arguments
    tool_output = selected_tool.invoke(tool_call["args"])

    # Append the tool output to the message list
    messages.append(ToolMessage(str(tool_output), tool_call_id=tool_call["id"]))

# Display the tool outputs appended to the conversation
for message in messages:
    print(message)


content='What is 3 * 12? Also, what is 11 + 49?' additional_kwargs={} response_metadata={}
content='' additional_kwargs={'tool_calls': [{'id': 'call_jzMBfdhav67wXBUDA92tZvb9', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_rNTbb5YcvCB2G8yTSVfgSWw8', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 88, 'total_tokens': 137, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-6796e3b5-b003-4d33-9c65-18199c8cb18e-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_jzMBfdhav67wXBUDA92tZvb9', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_rNTbb5

In [9]:
# Get the final response from the model with the tool results included
final_response = llm_with_tools.invoke(messages)
print("Final response from the model:", final_response.content)


Final response from the model: 3 * 12 is 36 and 11 + 49 is 60.


In [10]:
always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")
